In [1]:
import time
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from scikeras.wrappers import KerasClassifier

start_time = time.time()
# Set your data directory
data_directory = '/Users/shreyas/Desktop/5th_sem/DeepLearning_Lab/Project/training_experimental/data/train'  # Update this path to the directory containing your dataset

# Define constants
input_shape = (48, 48, 1)  # Updated input size to match your image size
num_classes = 7  # Automatically determine the number of classes
batch_size = 8192

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to the [0, 1] range
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and preprocess the data
datagen_with_preprocessing = datagen.flow_from_directory(
    data_directory,
    target_size=(48, 48),  # Updated target size
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

# Split the data into training and validation sets
X, y = datagen_with_preprocessing.next()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert y_train to one-hot encoding
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)

# Function to create the model
def create_model(optimizer='adam', loss='categorical_crossentropy', dropout_rate=0.2):
    model = keras.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(dropout_rate),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    return model

# Wrap the Keras model as a scikit-learn estimator
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define hyperparameter options
param_dist = {
    'optimizer': ['adam', 'rmsprop', 'sgd', 'adagrad'],
    'loss': ['categorical_crossentropy', 'kullback_leibler_divergence', 'squared_hinge', 'huber_loss'],
    'batch_size': [8192, 16384],
    'epochs': [20, 50, 100]
}

k_fold = KFold(n_splits=10)

# Perform the Randomized Search
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=10,  # Number of random parameter combinations to try
    cv=k_fold,  # Use MultiLabelKFold for multilabel data
    n_jobs=-1  # Use all available CPU cores for parallelism
)

# Perform the Randomized Search
random_search.fit(X_train, y_train)

# Get the best parameters and their accuracy
best_params = random_search.best_params_
best_accuracy = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy: {:.2f}%".format(best_accuracy * 100))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total training time: {elapsed_time:.2f} seconds")

Found 28709 images belonging to 7 classes.


PicklingError: Could not pickle the task to send it to the workers.

Found 28709 images belonging to 7 classes.
21/21 [==============================] - 0s 4ms/step


/Users/shreyas/Desktop/5th_sem/DeepLearning_Lab/Project/training_experimental/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
70 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/shreyas/Desktop/5th_sem/DeepLearning_Lab/Project/training_experimental/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/shreyas/Desktop/5th_sem/DeepLearning_Lab/Project/training_experimental/venv/lib/python3.11/site-packages/scikeras/wrappers.py", line 1491, in fit
    super(

205/205 [==============================] - 4s 18ms/step - loss: 1.8177 - accuracy: 0.2431


RandomizedSearchCV(cv=10,
                   estimator=KerasClassifier(model=<function create_model at 0x2c312a520>),
                   n_jobs=-1,
                   param_distributions={'loss': ['categorical_crossentropy',
                                                 'kullback_leibler_divergence',
                                                 'squared_hinge',
                                                 'huber_loss'],
                                        'optimizer': ['adam', 'rmsprop',
                                                      'sgd']})